In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

seed = 42

In [2]:
df = pd.read_csv("wine_quality.csv")

In [3]:
# df.describe()
# df.info()
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


In [4]:
y = df.pop("quality")

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=seed)


In [6]:
model = RandomForestRegressor(max_depth = 2, random_state = seed)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=42)

In [7]:
train_score = model.score(X_train, y_train)*100
print(train_score)

32.989576660267005


In [8]:
test_score = model.score(X_test, y_test)
print(test_score)

0.32035325871879183


In [9]:
with open("metrics.txt", 'w') as outfile:
    outfile.write("Training Variance: %2.1f%%\n" % train_score)
    outfile.write("Test Variance: %2.1f%%\n" % test_score)

In [10]:
# Feature Importance
importance = model.feature_importances_
labels = df.columns
feature_df = pd.DataFrame(list(zip(labels, importance)), columns = ["feature","importance"])
feature_df = feature_df.sort_values(by = 'importance', ascending = False)

In [11]:
# Image formatting Font size
axis_fs = 18 
title_fs = 22 
sns.set(style = "whitegrid")

# Feature Importance Plot
ax = sns.barplot(x="importance", y="feature", data=feature_df)
ax.set_xlabel('Importance',fontsize = axis_fs) 
ax.set_ylabel('Feature', fontsize = axis_fs)
ax.set_title('Random forest\nfeature importance', fontsize = title_fs)

plt.tight_layout()
plt.savefig("feature_importance.png",dpi=120) 
plt.close()

In [12]:
# Plot Residuals
# How far true qualities of predicted from Actual

y_pred = model.predict(X_test) + np.random.normal(0,0.25,len(y_test))
y_jitter = y_test + np.random.normal(0,0.25,len(y_test))
res_df = pd.DataFrame(list(zip(y_jitter,y_pred)), columns = ["true","pred"])


In [13]:
# Residual Plot

ax = sns.scatterplot(x="true", y="pred",data=res_df)
ax.set_aspect('equal')
ax.set_xlabel('True wine quality',fontsize = axis_fs) 
ax.set_ylabel('Predicted wine quality', fontsize = axis_fs)
ax.set_title('Residuals', fontsize = title_fs)

ax.plot([1, 10], [1, 10], 'black', linewidth = 1)
plt.ylim((2.5,8.5))
plt.xlim((2.5,8.5))

plt.tight_layout()
plt.savefig("residuals.png",dpi=120) 
plt.close()